Applied AI Services
You can use Cognitive Services to build your own AI solutions, and they also underpin Azure Applied AI Services that provide out-of-the-box solutions for common AI scenarios. Applied AI Services include:

1. Azure Form Recognizer - an optical character recognition (OCR) solution that can extract semantic meaning from forms, such as invoices, receipts, and others.<br>
2. Azure Metrics Advisor - A service build on the Anomaly Detector cognitive service that simplifies real-time monitoring and response to critical metrics.<br>
3. Azure Video Analyzer for Media - A comprehensive video analysts solution build on the Video Indexer cognitive service.<br>
4. Azure Immersive Reader - A reading solution that supports people of all ages and abilities.<br>
5. Azure Bot Service - A cloud service for delivering conversational AI solutions, or bots.<br>
6. Azure Cognitive Search - A cloud-scale search solution that uses cognitive services to extract insights from data and documents.

AI engineers can develop Bots by writing code, using the classes available in the Bot Framework SDK. Alternatively, you can use the Bot Framework Composer to develop complex bots using a visual design interface.

Azure Cognitive Search is an Applied AI Service that enables you to ingest and index data from various sources, and search the index to find, filter, and sort information extracted from the source data.

In addition to basic text-based indexing, Azure Cognitive Search enables you to define an enrichment pipeline that uses AI skills to enhance the index with insights derived from the source data - for example, by using computer vision and natural language processing capabilities to generate descriptions of images, extract text from scanned documents, and determine key phrases in large documents that encapsulate their key points.

Not only does this AI enrichment produce a more useful search experience, the insights extracted by your enrichment pipeline can be persisted in a knowledge store for further analysis or integration into a data pipeline for a business intelligence solution.

Options for Azure resources
For many of the available cognitive services, you can choose between the following provisioning options:

Multi-service resource
You can provision a Cognitive Services resource that supports multiple different cognitive services. For example, you could create a single resource that enables you to use the Language, Computer Vision, Speech, and other services.

This approach enables you to manage a single set of access credentials to consume multiple services at a single endpoint, and with a single point of billing for usage of all services.

Single-service resource
Each cognitive service can be provisioned individually, for example by creating discrete Language and Computer Vision resources in your Azure subscription.

This approach enables you to use separate endpoints for each service (for example to provision them in different geographical regions) and to manage access credentials for each service independently. It also enables you to manage billing separately for each service.

Single-service resources generally offer a free tier (with usage restrictions), making them a good choice to try out a service before using it in a production application.

<h3> Provisioning cognitive services</h3>

Start Visual Studio Code.

Open the palette (SHIFT+CTRL+P) and run a Git: Clone command to clone the https://github.com/MicrosoftLearning/AI-102-AIEngineer repository to a local folder (it doesn't matter which folder).

When the repository has been cloned, open the folder in Visual Studio Code.

Wait while additional files are installed to support the C# code projects in the repo.

<h3>Steps:</h3>
Open the Azure portal at https://portal.azure.com, and sign in using the Microsoft account associated with your Azure subscription.
Select the ＋Create a resource button, search for cognitive services, and create a Cognitive Services resource with the following settings:
Subscription: Your Azure subscription
Resource group: Choose or create a resource group (if you are using a restricted subscription, you may not have permission to create a new resource group - use the one provided)
Region: Choose any available region
Name: Enter a unique name
Pricing tier: Standard S0

In [2]:
#rest-client.py

from dotenv import load_dotenv
import os
import http.client, base64, json, urllib
from urllib import request, parse, error

def main():
    global cog_endpoint
    global cog_key

    try:
        # Get Configuration Settings
        load_dotenv()
        cog_endpoint = os.getenv('COG_SERVICE_ENDPOINT')
        cog_key = os.getenv('COG_SERVICE_KEY')

        # Get user input (until they enter "quit")
        userText =''
        while userText.lower() != 'quit':
            userText = input('Enter some text ("quit" to stop)\n')
            if userText.lower() != 'quit':
                GetLanguage(userText)


    except Exception as ex:
        print(ex)

def GetLanguage(text):
    try:
        # Construct the JSON request body (a collection of documents, each with an ID and text)
        jsonBody = {
            "documents":[
                {"id": 1,
                 "text": text}
            ]
        }

        # Let's take a look at the JSON we'll send to the service
        print(json.dumps(jsonBody, indent=2))

        # Make an HTTP request to the REST interface
        uri = cog_endpoint.rstrip('/').replace('https://', '')
        conn = http.client.HTTPSConnection(uri)

        # Add the authentication key to the request header
        headers = {
            'Content-Type': 'application/json',
            'Ocp-Apim-Subscription-Key': cog_key
        }

        # Use the Text Analytics language API
        conn.request("POST", "/text/analytics/v3.1/languages?", str(jsonBody).encode('utf-8'), headers)

        # Send the request
        response = conn.getresponse()
        data = response.read().decode("UTF-8")

        # If the call was successful, get the response
        if response.status == 200:

            # Display the JSON response in full (just so we can see it)
            results = json.loads(data)
            print(json.dumps(results, indent=2))

            # Extract the detected language name for each document
            for document in results["documents"]:
                print("\nLanguage:", document["detectedLanguage"]["name"])

        else:
            # Something went wrong, write the whole response
            print(data)

        conn.close()


    except Exception as ex:
        print(ex)


if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'dotenv'

<h3> Using SDK to consume cognitive service</h3>

Execute: pip install azure-ai-textanalytics==5.1.0

from dotenv import load_dotenv
import os
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

def main():
    global cog_endpoint
    global cog_key

    try:
        # Get Configuration Settings
        load_dotenv()
        cog_endpoint = os.getenv('COG_SERVICE_ENDPOINT')
        cog_key = os.getenv('COG_SERVICE_KEY')

        # Get user input (until they enter "quit")
        userText =''
        while userText.lower() != 'quit':
            userText = input('\nEnter some text ("quit" to stop)\n')
            if userText.lower() != 'quit':
                language = GetLanguage(userText)
                print('Language:', language)

    except Exception as ex:
        print(ex)
    
def GetLanguage(text):

    # Create client using endpoint and key
    credential = AzureKeyCredential(cog_key)
    client = TextAnalyticsClient(endpoint=cog_endpoint, credential=credential)

    # Call the service to get the detected language
    detectedLanguage = client.detect_language(documents = [text])[0]
    return detectedLanguage.primary_language.name


if __name__ == "__main__":
    main()

<h3> Secure congitve services</h3>

You can store the subscription keys for a cognitive services resource in Azure Key Vault, and assign a managed identity to client applications that need to use the service. The applications can then retrieve the key as needed from the key vault, without risk of exposing it to unauthorized users.

<h3>Token-based authentication</h3>

When using the REST interface, some Cognitive Services support (or even require) token-based authentication. In these cases, the subscription key is presented in an initial request to obtain an authentication token, which has a valid period of 10 minutes. Subsequent requests must present the token to validate that the caller has been authenticated.

<h3>Azure Active Directory authentication</h3>

Some Cognitive Services support Azure Active Directory authentication, enabling you to grant access to specific service principals or managed identities for apps and services running in Azure

By default, Cognitive Services are accessible from all networks. Some individual Cognitive Services resources (such as Text Analytics, Face, Computer Vision, and others) can be configured to restrict access to specific network addresses - either public Internet addresses or addresses on virtual networks.

In [4]:
#keyvault-client.py

from dotenv import load_dotenv
import os
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
from azure.keyvault.secrets import SecretClient
from azure.identity import ClientSecretCredential


def main():
    global cog_endpoint
    global cog_key

    try:
        # Get Configuration Settings
        load_dotenv()
        cog_endpoint = os.getenv('COG_SERVICE_ENDPOINT')
        key_vault_name = os.getenv('KEY_VAULT')
        app_tenant = os.getenv('TENANT_ID')
        app_id = os.getenv('APP_ID')
        app_password = os.getenv('APP_PASSWORD')

        # Get cognitive services key from keyvault using the service principal credentials
        key_vault_uri = f"https://{key_vault_name}.vault.azure.net/"
        credential = ClientSecretCredential(app_tenant, app_id, app_password)
        keyvault_client = SecretClient(key_vault_uri, credential)
        secret_key = keyvault_client.get_secret("Cognitive-Services-Key")
        cog_key = secret_key.value

        # Get user input (until they enter "quit")
        userText =''
        while userText.lower() != 'quit':
            userText = input('\nEnter some text ("quit" to stop)\n')
            if userText.lower() != 'quit':
                language = GetLanguage(userText)
                print('Language:', language)

    except Exception as ex:
        print(ex)

def GetLanguage(text):

    # Create client using endpoint and key
    credential = AzureKeyCredential(cog_key)
    client = TextAnalyticsClient(endpoint=cog_endpoint, credential=credential)

    # Call the service to get the detected language
    detectedLanguage = client.detect_language(documents = [text])[0]
    return detectedLanguage.primary_language.name


if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'dotenv'

<h3>Monitor Cognitive Service</h3>

<h3>Monitor cost</h3>

Plan costs for Cognitive Services

Before deploying a solution that depends on Cognitive Services, you can estimate costs by using the Azure Pricing Calculator.

To view costs for Cognitive Services, sign into the Azure portal and select your subscription. You can then view overall costs for the subscription by selecting the Cost analysis tab. To view only costs for Cognitive Services, add a filter that restricts the data to reflect resources with a service name of azure cognitive services.

<h3>Alert rules</h3>
To create an alert rule for a Cognitive Services resource, select the resource in the Azure portal and on the Alerts tab, add a new alert rule. To define the alert rule, you must specify:

The scope of the alert rule - in other words, the resource you want to monitor.
A condition on which the alert is triggered. The specific trigger for the alert is based on a signal type, which can be Activity Log (an entry in the activity log created by an action performed on the resource, such as regenerating its subscription keys) or Metric (a metric threshold such as the number of errors exceeding 10 in an hour).
Optional actions, such as sending an email to an administrator notifying them of the alert, or running an Azure Logic App to address the issue automatically.
Alert rule details, such as a name for the alert rule and the resource group in which it should be defined.

You can view metrics for an individual resource in the Azure portal by selecting the resource and viewing its Metrics page. On this page, you can add resource-specific metrics to charts. By default an empty chart is created for you, and you can add more charts as required.

For example, the following image shows the Metrics page for an Azure Cognitive Services resource, showing the count of total calls to the service over a period of time.

In the Azure portal, you can create dashboards that consist of multiple visualizations from different resources in your Azure environment to help you gain an overall view of the health and performance of your Azure resources.

To create a dashboard, select Dashboard in the Azure portal menu (your default view may already be set to a dashboard rather than the portal home page). From here, you can add up to 100 named dashboards to encapsulate views for specific aspects of your Azure services that you want to track.

<h3>Manage diagnostic logging</h3>

To capture diagnostic logs for a Cognitive Services resource, you need a destination for the log data. You can use Azure Event Hub as a destination in order to then forward the data on to a custom telemetry solution, and you can connect directly to some third-party solutions; but in most cases you'll use one (or both) of the following kinds of resource within your Azure subscription:

    Azure Log Analytics - a service that enables you to query and visualize log data within the Azure portal.
    Azure Storage - a cloud-based data store that you can use to store log archives (which can be exported for analysis in other tools as needed).
You should create these resources before configuring diagnostic logging for your Cognitive Services resource. If you intend to archive log data to Azure Storage, create the Azure Storage account in the same region as your Cognitive Services resource.




<h3> Use Cognitive Services containers</h3>

To deploy and use a Cognitive Services container, the following three activities must occur:

The container image for the specific Cognitive Services API you want to use is downloaded and deployed to a container host, such as a local Docker server, an Azure Container Instance (ACI), or Azure Kubernetes Service (AKS).
Client applications submit data to the endpoint provided by the containerized service, and retrieve results just as they would from a Cognitive Services cloud resource in Azure.
Periodically, usage metrics for the containerized service are sent to a Cognitive Services resource in Azure in order to calculate billing for the service.

Even when using a container, you must provision a Cognitive Services resource in Azure for billing purposes

When you deploy a Cognitive Services container image to a host, you must specify three settings.

Setting	Description<br>
ApiKey	Key from your deployed Azure Cognitive Service; used for billing.<br>
Billing	Endpoint URI from your deployed Azure Cognitive Service; used for billing.<br>
Eula	Value of accept to state you accept the license for the container.

command to run container cognitive service

curl -X POST "http://<your_ACI_IP_address_or_FQDN>:5000/text/analytics/v3.0/languages?" -H "Content-Type: application/json" --data-ascii "{'documents':[{'id':1,'text':'Hello world.'},{'id':2,'text':'Salut tout le monde.'}]}"


<h2> Process and translate text with ACS </h2>

The Language service is designed to help you extract information from text. It provides functionality that you can use for:

1. Language detection - determining the language in which text is written.
2. Key phrase extraction - identifying important words and phrases in the text that indicate the main points.
3. Sentiment analysis - quantifying how positive or negative the text is.
4. Named entity recognition - detecting references to entities, including people, locations, time periods, organizations, and more.
5. Entity linking - identifying specific entities by providing reference links to Wikipedia articles.

To use the Language service to analyze text, you must provision a resource for it in your Azure subscription. You can provision a single-service Language resource, or you can use a multi-service Cognitive Services resource.

The Language service recognizes up to 120 languages.

Language detection can work with documents or single phrases. It's important to note that the document size must be under 5,120 characters. The size limit is per document and each collection is restricted to 1,000 items (IDs). A sample of a properly formatted JSON payload that you might submit to the service in the request body is shown here, including a collection of documents, each containing a unique id and the text to be analyzed. Optionally, you can provide a countryHint to improve prediction performance.

In [5]:
{
  "documents": [
    {
      "countryHint": "US",
      "id": "1",
      "text": "Hello world"
    },
    {
      "id": "2",
      "text": "Bonjour tout le monde"
    }
  ]
}

{'documents': [{'countryHint': 'US', 'id': '1', 'text': 'Hello world'},
  {'id': '2', 'text': 'Bonjour tout le monde'}]}

The service will return a JSON response that contains a result for each document in the request body

In [6]:
{
  "documents": [
   {
     "id": "1",
     "detectedLanguage": {
       "name": "English",
       "iso6391Name": "en",
       "confidenceScore": 1
     },
     "warnings": []
   },
   {
     "id": "2",
     "detectedLanguage": {
       "name": "French",
       "iso6391Name": "fr",
       "confidenceScore": 1
     },
     "warnings": []
   }
  ],
  "errors": [],
  "modelVersion": "2020-04-01"
}

{'documents': [{'id': '1',
   'detectedLanguage': {'name': 'English',
    'iso6391Name': 'en',
    'confidenceScore': 1},
   'warnings': []},
  {'id': '2',
   'detectedLanguage': {'name': 'French',
    'iso6391Name': 'fr',
    'confidenceScore': 1},
   'warnings': []}],
 'errors': [],
 'modelVersion': '2020-04-01'}

 Mixed language content within the same document returns the language with the largest representation in the content, but with a lower positive rating, reflecting the marginal strength of that assessment.

In [7]:
{
  "documents": [
    {
      "id": "1",
      "text": "Hello, I would like to take a class at your University. ¿Se ofrecen clases en español? Es mi primera lengua y más fácil para escribir. Que diriez-vous des cours en français?"
    }
  ]
}

{'documents': [{'id': '1',
   'text': 'Hello, I would like to take a class at your University. ¿Se ofrecen clases en español? Es mi primera lengua y más fácil para escribir. Que diriez-vous des cours en français?'}]}

In [8]:
{
  "documents": [
    {
      "id": "1",
      "detectedLanguages": [
        {
          "name": "Spanish",
          "iso6391Name": "es",
          "score": 0.9375
        }
      ]
    }
  ],
  "errors": []
}

{'documents': [{'id': '1',
   'detectedLanguages': [{'name': 'Spanish',
     'iso6391Name': 'es',
     'score': 0.9375}]}],
 'errors': []}

The last condition to consider is when there is ambiguity as to the language content. The scenario might happen if you submit textual content that the analyzer is not able to parse, for example because of character encoding issues when converting the text to a string variable. As a result, the response for the language name and ISO code will indicate (unknown) and the score value will be returned as NaN, or Not a Number.

<h3>Extract key phrases</h3>

Key phrase extraction is the process of evaluating the text of a document, or documents, and then identifying the main points around the context of the document(s).

In [10]:
{
  "documents": [
    {
      "id": "1",
      "language": "en",
      "text": "You must be the change you wish to see in the world."
    },
    {
      "id": "2",
      "language": "en",
      "text": "The journey of a thousand miles begins with a single step."
    }
  ]
}

{'documents': [{'id': '1',
   'language': 'en',
   'text': 'You must be the change you wish to see in the world.'},
  {'id': '2',
   'language': 'en',
   'text': 'The journey of a thousand miles begins with a single step.'}]}

In [11]:
{
  "documents": [
   {
     "id": "1",
     "keyPhrases": [
       "change",
       "world"
     ],
     "warnings": []
   },
   {
     "id": "2",
     "keyPhrases": [
       "miles",
       "single step",
       "journey"
     ],
     "warnings": []
   }
  ],
  "errors": [],
  "modelVersion": "2020-04-01"
}

{'documents': [{'id': '1', 'keyPhrases': ['change', 'world'], 'warnings': []},
  {'id': '2',
   'keyPhrases': ['miles', 'single step', 'journey'],
   'warnings': []}],
 'errors': [],
 'modelVersion': '2020-04-01'}

<h3>Analyze Sentiment</h3>

When using the Language service to evaluate sentiment, the response includes overall document sentiment and individual sentence sentiment for each document submitted to the service.

In [12]:
{
  "documents": [
    {
      "language": "en",
      "id": "1",
      "text": "Smile! Life is good!"
    }
  ]
}

{'documents': [{'language': 'en', 'id': '1', 'text': 'Smile! Life is good!'}]}

In [13]:
{
  "documents": [
   {
     "id": "1",
     "sentiment": "positive",
     "confidenceScores": {
       "positive": 0.99,
       "neutral": 0.01,
       "negative": 0.00
     },
     "sentences": [
       {
         "text": "Smile!",
         "sentiment": "positive",
         "confidenceScores": {   
             "positive": 0.97,
	         "neutral": 0.02, 
             "negative": 0.01
           },
         "offset": 0,
         "length": 6
       },
       {
	      "text": "Life is good!",
          "sentiment": "positive",
          "confidenceScores": {   
             "positive": 0.98,
	         "neutral": 0.02,  
             "negative": 0.00
           },
         "offset": 7,
         "length": 13
       }
     ],
     "warnings": []
   }
  ],
  "errors": [],
  "modelVersion": "2020-04-01"
}

{'documents': [{'id': '1',
   'sentiment': 'positive',
   'confidenceScores': {'positive': 0.99, 'neutral': 0.01, 'negative': 0.0},
   'sentences': [{'text': 'Smile!',
     'sentiment': 'positive',
     'confidenceScores': {'positive': 0.97, 'neutral': 0.02, 'negative': 0.01},
     'offset': 0,
     'length': 6},
    {'text': 'Life is good!',
     'sentiment': 'positive',
     'confidenceScores': {'positive': 0.98, 'neutral': 0.02, 'negative': 0.0},
     'offset': 7,
     'length': 13}],
   'warnings': []}],
 'errors': [],
 'modelVersion': '2020-04-01'}

Overall document sentiment is based on sentences:

    If all sentences are neutral, the overall sentiment is neutral.
    If sentence classifications include only positive and neutral, the overall sentiment is positive.
    If the sentence classifications include only negative and neutral, the overall sentiment is negative.
    If the sentence classifications include positive and negative, the overall sentiment is mixed.

<h3>Extract entities</h3>

Named Entity Recognition identifies entities that are mentioned in the text. Entities are grouped into categories and subcategories, for example:

    Person
    Location
    DateTime
    Organization
    Address
    Email
    URL

In [14]:
{
  "documents": [
    {
      "language": "en",
      "id": "1",
      "text": "Joe went to London on Saturday"
    }
  ]
}

{'documents': [{'language': 'en',
   'id': '1',
   'text': 'Joe went to London on Saturday'}]}

In [15]:
{
  "documents":[
      {
          "id":"1",
          "entities":[
          {
            "text":"Joe",
            "category":"Person",
            "offset":0,
            "length":3,
            "confidenceScore":0.62
          },
          {
            "text":"London",
            "category":"Location",
            "subcategory":"GPE",
            "offset":12,
            "length":6,
            "confidenceScore":0.88
          },
          {
            "text":"Saturday",
            "category":"DateTime",
            "subcategory":"Date",
            "offset":22,
            "length":8,
            "confidenceScore":0.8
          }
        ],
        "warnings":[]
      }
  ],
  "errors":[],
  "modelVersion":"2021-01-15"
}

{'documents': [{'id': '1',
   'entities': [{'text': 'Joe',
     'category': 'Person',
     'offset': 0,
     'length': 3,
     'confidenceScore': 0.62},
    {'text': 'London',
     'category': 'Location',
     'subcategory': 'GPE',
     'offset': 12,
     'length': 6,
     'confidenceScore': 0.88},
    {'text': 'Saturday',
     'category': 'DateTime',
     'subcategory': 'Date',
     'offset': 22,
     'length': 8,
     'confidenceScore': 0.8}],
   'warnings': []}],
 'errors': [],
 'modelVersion': '2021-01-15'}

<h3>Extract linked entities</h3>

Entity linking can be used to disambiguate entities of the same name by referencing an article in a knowledge base. Wikipedia provides the knowledge base for the Text Analytics service. Specific article links are determined based on entity context within the text.

In [16]:
{
  "documents": [
    {
      "language": "en",
      "id": "1",
      "text": "I saw Venus shining in the sky"
    }
  ]
}

{'documents': [{'language': 'en',
   'id': '1',
   'text': 'I saw Venus shining in the sky'}]}

In [17]:
{
  "documents":
    [
      {
        "id":"1",
        "entities":[
          {
            "name":"Venus",
            "matches":[
              {
                "text":"Venus",
                "offset":6,
                "length":5,
                "confidenceScore":0.01
              }
            ],
            "language":"en",
            "id":"Venus",
            "url":"https://en.wikipedia.org/wiki/Venus",
            "dataSource":"Wikipedia"
          }
        ],
        "warnings":[]
      }
    ],
  "errors":[],
  "modelVersion":"2020-02-01"
}

{'documents': [{'id': '1',
   'entities': [{'name': 'Venus',
     'matches': [{'text': 'Venus',
       'offset': 6,
       'length': 5,
       'confidenceScore': 0.01}],
     'language': 'en',
     'id': 'Venus',
     'url': 'https://en.wikipedia.org/wiki/Venus',
     'dataSource': 'Wikipedia'}],
   'warnings': []}],
 'errors': [],
 'modelVersion': '2020-02-01'}

pip install azure-ai-textanalytics==5.1.0


In [18]:
from dotenv import load_dotenv
import os

# Import namespaces
# import namespaces
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

def main():
    try:
        # Get Configuration Settings
        load_dotenv()
        cog_endpoint = os.getenv('COG_SERVICE_ENDPOINT')
        cog_key = os.getenv('COG_SERVICE_KEY')

        # Create client using endpoint and key
        # Create client using endpoint and key
        credential = AzureKeyCredential(cog_key)
        cog_client = TextAnalyticsClient(endpoint=cog_endpoint, credential=credential)

        # Analyze each text file in the reviews folder
        reviews_folder = 'reviews'
        for file_name in os.listdir(reviews_folder):
            # Read the file contents
            print('\n-------------\n' + file_name)
            text = open(os.path.join(reviews_folder, file_name), encoding='utf8').read()
            print('\n' + text)

            # Get language
            # Get language
            detectedLanguage = cog_client.detect_language(documents=[text])[0]
            print('\nLanguage: {}'.format(detectedLanguage.primary_language.name))

            # Get sentiment
            # Get sentiment
            sentimentAnalysis = cog_client.analyze_sentiment(documents=[text])[0]
            print("\nSentiment: {}".format(sentimentAnalysis.sentiment))

            # Get key phrases
            # Get key phrases
            phrases = cog_client.extract_key_phrases(documents=[text])[0].key_phrases
            if len(phrases) > 0:
                print("\nKey Phrases:")
                for phrase in phrases:
                    print('\t{}'.format(phrase))

            # Get entities
            # Get entities
            entities = cog_client.recognize_entities(documents=[text])[0].entities
            if len(entities) > 0:
                print("\nEntities")
                for entity in entities:
                    print('\t{} ({})'.format(entity.text, entity.category))

            # Get linked entities
            # Get linked entities
            entities = cog_client.recognize_linked_entities(documents=[text])[0].entities
            if len(entities) > 0:
                print("\nLinks")
                for linked_entity in entities:
                    print('\t{} ({})'.format(linked_entity.name, linked_entity.url))


    except Exception as ex:
        print(ex)


if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'dotenv'

<h3> Translator Azure Cognitive Service</h3>

The Translator service provides a multilingual text translation API that you can use for:

    Language detection
    One-to-many translation
    Script transliteration (converting text from its native script to an alternative script)
    
You can provision a single-service Translator resource, or you can use the Text Analytics API in a multi-service Cognitive Services resource.

<h3>Language detection</h3>

Use detect REST function to detect the language

In [19]:
{ 'Text' : 'こんにちは' }

{'Text': 'こんにちは'}

In [20]:
[
  {
    "isTranslationSupported": true,
    "isTransliterationSupported": true,
    "language": "ja",
    "score": 1.0
   }
]

NameError: name 'true' is not defined

<h3>Translation</h3>

 use the translate function; specifying a single from parameter to indicate the source language, and one or more to parameters to specify the languages into which you want the text translated.

In [21]:
[
  {"translations": 
    [
      {"text": "Hello", "to": "en"},   
      {"text": "Bonjour", "to": "fr"}
    ]
  }
]

[{'translations': [{'text': 'Hello', 'to': 'en'},
   {'text': 'Bonjour', 'to': 'fr'}]}]

<h3>Transliteration</h3>

To accomplish this, we can submit the Japanese text to the transliterate function with a fromScript parameter of Jpan and a toScript parameter of Latn to get the following result:

In [22]:
[
    {
        "script": "Latn",
        "text": "Kon'nichiwa"
    }
]

[{'script': 'Latn', 'text': "Kon'nichiwa"}]

<h3>Word alignment</h3>

In written English (using Latin script), spaces are used to separate words. However, in some other languages (and more specifically, scripts) this is not always the case.

To resolve this problem, you can specify the includeAlignment parameter with a value of true to produce the following result:

In [23]:
[
  {"translations": 
    [
      {"text": "认知服务", "to": "zh-Hans",
       "alignment": {"proj": "0:8-0:1 10:17-2:3"}
      }
    ]
  }
]

[{'translations': [{'text': '认知服务',
    'to': 'zh-Hans',
    'alignment': {'proj': '0:8-0:1 10:17-2:3'}}]}]

These results tell us that characters 0 to 8 in the source correspond to characters 0 to 1 in the translation (), while characters 10 to 17 in the source correspond to characters 2 to 3 in the translation.

<h3>Sentence length</h3>

Sometimes it might be useful to know the length of a translation, for example to determine how best to display it in a user interface. You can get this information by setting the includeSentenceLength parameter to true

<h3>Profanity filtering</h3>


Sometimes text contains profanities, which you might want to obscure or omit altogether in a translation. You can handle profanities by specifying the profanityAction parameter, which can have one of the following values:

    NoAction: Profanities are translated along with the rest of the text.
    Deleted: Profanities are omitted in the translation.
    Marked: Profanities are indicated using the technique indicated in the profanityMarker parameter (if supplied). The default value for this parameter is Asterisk, which replaces characters in profanities with "*". As an alternative, you can specify a profanityMarker value of Tag, which causes profanities to be enclosed in XML tags.

<h3>Define custom translations</h3>

To solve this problem, you can create a custom model that maps your own sets of source and target terms for translation. To create a custom model, use the Custom Translator portal to:

    Create a workspace linked to your Translator resource
    Create a project
    Upload training data files
    Train a model

Your custom model is assigned a unique category Id, which you can specify in translate calls to your Translator resource by using the category parameter, causing translation to be performed by your custom model instead of the default model.

In [26]:
from dotenv import load_dotenv
import os
import requests, json

def main():
    global translator_endpoint
    global cog_key
    global cog_region

    try:
        # Get Configuration Settings
        load_dotenv()
        cog_key = os.getenv('COG_SERVICE_KEY')
        cog_region = os.getenv('COG_SERVICE_REGION')
        translator_endpoint = 'https://api.cognitive.microsofttranslator.com'

        # Analyze each text file in the reviews folder
        reviews_folder = 'reviews'
        for file_name in os.listdir(reviews_folder):
            # Read the file contents
            print('\n-------------\n' + file_name)
            text = open(os.path.join(reviews_folder, file_name), encoding='utf8').read()
            print('\n' + text)

            # Detect the language
            language = GetLanguage(text)
            print('Language:',language)

            # Translate if not already English
            if language != 'en':
                translation = Translate(text, language)
                print("\nTranslation:\n{}".format(translation))
                
    except Exception as ex:
        print(ex)

def GetLanguage(text):
    # Default language is English
    language = 'en'

    # Use the Translator detect function
    # Use the Translator detect function
    path = '/detect'
    url = translator_endpoint + path

    # Build the request
    params = {
        'api-version': '3.0'
    }

    headers = {
    'Ocp-Apim-Subscription-Key': cog_key,
    'Ocp-Apim-Subscription-Region': cog_region,
    'Content-type': 'application/json'
    }

    body = [{
        'text': text
    }]

    # Send the request and get response
    request = requests.post(url, params=params, headers=headers, json=body)
    response = request.json()

    # Parse JSON array and get language
    language = response[0]["language"]

    # Return the language
    return language

def Translate(text, source_language):
    translation = ''

    # Use the Translator translate function
    # Use the Translator translate function
    path = '/translate'
    url = translator_endpoint + path

    # Build the request
    params = {
        'api-version': '3.0',
        'from': source_language,
        'to': ['en']
    }

    headers = {
        'Ocp-Apim-Subscription-Key': cog_key,
        'Ocp-Apim-Subscription-Region': cog_region,
        'Content-type': 'application/json'
    }

    body = [{
        'text': text
    }]

    # Send the request and get response
    request = requests.post(url, params=params, headers=headers, json=body)
    response = request.json()

    # Parse JSON array and get translation
    translation = response[0]["translations"][0]["text"]

    # Return the translation
    return translation

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'dotenv'

<h3>Speech enabled apps with speech service</h3>

The Speech service provides APIs that you can use to build speech-enabled applications. Specifically, the Speech service supports:

    Speech-to-Text: An API that enables speech recognition in which your application can accept spoken input.
    Text-to-Speech: An API that enables speech synthesis in which your application can provide spoken output.
    Speech Translation: An API that you can use to translate spoken input into multiple languages.
    Speaker Recognition: An API that enables your application to recognize individual speakers based on their voice.
    Intent Recognition: An API that integrates with the Language Understanding service to determine the semantic meaning of spoken input.

 You can use either a dedicated Speech resource or a multi-service Cognitive Services resource.
 
 After creating your Azure resource, you will need the following information to use it from a client application through one of the supported SDKs:

The location in which the resource is deployed (for example, eastus)
One of the keys assigned to your resource.

The Speech service supports speech recognition through two REST APIs:

    The Speech-to-text API, which is the primary way to perform speech recognition.
    The Speech-to-text Short Audio API, which is optimized for short streams of audio (up to 60 seconds).


<h3>Using the Speech-to-text SDK</h3>

1. Use a SpeechConfig object to encapsulate the information required to connect to your Speech resource. Specifically, its location and key.
2. Optionally, use an AudioConfig to define the input source for the audio to be transcribed. By default, this is the default system microphone, but you can also specify an audio file.
3. Use the SpeechConfig and AudioConfig to create a SpeechRecognizer object. This object is a proxy client for the Speech-to-text API.
4. Use the methods of the SpeechRecognizer object to call the underlying API functions. For example, the RecognizeOnceAsync() method uses the Speech service to asynchronously transcribe a single spoken utterance.
4. Process the response from the Speech service. In the case of the RecognizeOnceAsync() method, the result is a 
5. SpeechRecognitionResult object that includes the following properties:
        Duration
        OffsetInTicks
        Properties
        Reason
        ResultId
        Text
        
If the operation was successful, the Reason property has the enumerated value RecognizedSpeech, and the Text property contains the transcription. Other possible values for Result include NoMatch (indicating that the audio was successfully parsed but no speech was recognized) or Canceled, indicating that an error occurred (in which case, you can check the Properties collection for the CancellationReason property to determine what went wrong.)

<h3>Use the text-to-speech API</h3>

Similarly to Speech-to-text APIs, the Speech service offers two REST APIs for speech synthesis:

    The Text-to-speech API, which is the primary way to perform speech synthesis.
    The Text-to-speech Long Audio API, which is designed to support batch operations that convert large volumes of text to audio - for example to generate an audio-book from the source text.
    
<h3>Using the Text-to-speech SDK</h3>

1. Use a SpeechConfig object to encapsulate the information required to connect to your Speech resource. Specifically, its location and key.
2. Optionally, use an AudioConfig to define the output device for the speech to be synthesized. By default, this is the default system speaker, but you can also specify an audio file, or by explicitly setting this value to a null value, you can process the audio stream object that is returned directly.
3. Use the SpeechConfig and AudioConfig to create a SpeechSynthesizer object. This object is a proxy client for the Text-to-speech API.
4. Use the methods of the SpeechSynthesizer object to call the underlying API functions. For example, the SpeakTextAsync() method uses the Speech service to convert text to spoken audio.
5. Process the response from the Speech service. In the case of the SpeakTextAsync method, the result is a SpeechSynthesisResult object that contains the following properties:
        AudioData
        Properties
        Reason
        ResultId
When speech has been successfully synthesized, the Reason property is set to the SynthesizingAudioCompleted enumeration and the AudioData property contains the audio stream (which, depending on the AudioConfig may have been automatically sent to a speaker or file).


<h3>Audio format</h3>

The Speech service supports multiple output formats for the audio stream that is generated by speech synthesis. Depending on your specific needs, you can choose a format based on the required:

    Audio file type
    Sample-rate
    Bit-depth

To specify the required output format, use the SetSpeechSynthesisOutputFormat method of the SpeechConfig object:

speechConfig.SetSpeechSynthesisOutputFormat(SpeechSynthesisOutputFormat.Riff24Khz16BitMonoPcm);


<h3>Voices</h3>

The Speech service provides multiple voices that you can use to personalize your speech-enabled applications. There are two kinds of voice that you can use:

    Standard voices - synthetic voices created from audio samples.
    Neural voices - more natural sounding voices created using deep neural networks.
Voices are identified by names that indicate a locale and a person's name - for example en-GB-George.

To specify a voice for speech synthesis in the SpeechConfig, set its SpeechSynthesisVoiceName property to the voice you want to use:

speechConfig.SpeechSynthesisVoiceName = "en-GB-George";

<h3>Use Speech Synthesis Markup Language</h3>

This SSML specifies a spoken dialog between two different neural voices, like this:

    Ariana (cheerfully): "I say tomato:
    Guy: "I say tomato (pronounced tom-ah-toe) ... Let's call the whole thing off!"

To submit an SSML description to the Speech service, you can use the SpeakSsmlAsync() method, like this:

speechSynthesizer.SpeakSsmlAsync(ssml_string);

In [2]:
#speaking-clock.py


from dotenv import load_dotenv
from datetime import datetime
import os

# Import namespaces
# Import namespaces
import azure.cognitiveservices.speech as speech_sdk
from playsound import playsound

def main():
    try:
        global speech_config

        # Get Configuration Settings
        load_dotenv()
        cog_key = os.getenv('COG_SERVICE_KEY')
        cog_region = os.getenv('COG_SERVICE_REGION')

        # Configure speech service
        # Configure speech service
        speech_config = speech_sdk.SpeechConfig(cog_key, cog_region)
        print('Ready to use speech service in:', speech_config.region)

        # Get spoken input
        command = TranscribeCommand()
        if command.lower() == 'what time is it?':
            TellTime()

    except Exception as ex:
        print(ex)

def TranscribeCommand():
    command = ''

    # Configure speech recognition
# Configure speech recognition
    audioFile = 'time.wav'
    playsound(audioFile)
    audio_config = speech_sdk.AudioConfig(filename=audioFile)
    speech_recognizer = speech_sdk.SpeechRecognizer(speech_config, audio_config)

    # Process speech input
    # Process speech input
    speech = speech_recognizer.recognize_once_async().get()
    if speech.reason == speech_sdk.ResultReason.RecognizedSpeech:
        command = speech.text
        print(command)
    else:
        print(speech.reason)
        if speech.reason == speech_sdk.ResultReason.Canceled:
            cancellation = speech.cancellation_details
            print(cancellation.reason)
            print(cancellation.error_details)

    # Return the command
    return command


def TellTime():
    now = datetime.now()
    response_text = 'The time is {}:{:02d}'.format(now.hour,now.minute)


    # Configure speech synthesis
    # Configure speech synthesis
    speech_config.speech_synthesis_voice_name = "en-GB-RyanNeural"
    speech_synthesizer = speech_sdk.SpeechSynthesizer(speech_config)

    # Synthesize spoken output
    # Synthesize spoken output
    speak = speech_synthesizer.speak_text_async(response_text).get()
    if speak.reason != speech_sdk.ResultReason.SynthesizingAudioCompleted:
        print(speak.reason)

    # Print the response
    print(response_text)


if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'dotenv'

<h3>Speech Translation</h3>

Before you can use the Speech service, you need to create a Speech resource in your Azure subscription. You can use either a dedicated Speech resource or a multi-service Cognitive Services resource.

<h3>Translate speech to text</h3>

1. Use a SpeechConfig object to encapsulate the information required to connect to your Speech resource. Specifically, its location and key.
2. Use a SpeechTranslationConfig object to specify the speech recognition language (the language in which the input speech is spoken) and the target languages into which it should be translated.
Optionally, use an AudioConfig to define the input source for the audio to be transcribed. By default, this is the default system microphone, but you can also specify an audio file.
3. Use the SpeechConfig, SpeechTranslationConfig, and AudioConfig to create a TranslationRecognizer object. This object is a proxy client for the Speech service translation API.
4. Use the methods of the TranslationRecognizer object to call the underlying API functions. For example, the RecognizeOnceAsync() method uses the Speech service to asynchronously translate a single spoken utterance.
5. Process the response from the Speech service. In the case of the RecognizeOnceAsync() method, the result is a SpeechRecognitionResult object that includes the following properties:
        Duration
        OffsetInTicks
        Properties
        Reason
        ResultId
        Text
        Translations
If the operation was successful, the Reason property has the enumerated value RecognizedSpeech, the Text property contains the transcription in the original language, and the Translations property contains a dictionary of the translations (using the two-character ISO language code, such as "en" for English, as a key).

<h3>Synthesize translations</h3>

The TranslationRecognizer returns translated transcriptions of spoken input - essentially translating audible speech to text.

You can also synthesize the translation as speech to create speech-to-speech translation solutions. There are two ways you can accomplish this.

Event-based synthesis

When you want to perform 1:1 translation (translating from one source language into a single target language), you can use event-based synthesis to capture the translation as an audio stream. To do this, you need to:

1. Specify the desired voice for the translated speech in the TranslationConfig.
2. Create an event handler for the TranslationRecognizer object's Synthesizing event.
3. In the event handler, use the GetAudio() method of the Result parameter to retrieve the byte stream of translated audio.

Manual synthesis

Manual synthesis is an alternative approach to event-based synthesis that doesn't require you to implement an event handler. You can use manual synthesis to generate audio translations for one or more target languages.

Manual synthesis of translations is essentially just the combination of two separate operations in which you:

1. Use a TranslationRecognizer to translate spoken input into text transcriptions in one or more target languages.
2. Iterate through the Translations dictionary in the result of the translation operation, using a SpeechSynthesizer to synthesize an audio stream for each language.

pip install azure-cognitiveservices-speech==1.19.0


pip install playsound==1.2.2

In [2]:
#translator.py


from dotenv import load_dotenv
from datetime import datetime
import os

# Import namespaces
# Import namespaces
import azure.cognitiveservices.speech as speech_sdk
from playsound import playsound
def main():
    try:
        global speech_config
        global translation_config

        # Get Configuration Settings
        load_dotenv()
        cog_key = os.getenv('COG_SERVICE_KEY')
        cog_region = os.getenv('COG_SERVICE_REGION')

        # Configure translation
        # Configure translation
        translation_config = speech_sdk.translation.SpeechTranslationConfig(cog_key, cog_region)
        translation_config.speech_recognition_language = 'en-US'
        translation_config.add_target_language('fr')
        translation_config.add_target_language('es')
        translation_config.add_target_language('hi')
        print('Ready to translate from',translation_config.speech_recognition_language)

        # Configure speech
        # Configure speech
        speech_config = speech_sdk.SpeechConfig(cog_key, cog_region)

        # Get user input
        targetLanguage = ''
        while targetLanguage != 'quit':
            targetLanguage = input('\nEnter a target language\n fr = French\n es = Spanish\n hi = Hindi\n Enter anything else to stop\n').lower()
            if targetLanguage in translation_config.target_languages:
                Translate(targetLanguage)
            else:
                targetLanguage = 'quit'
                

    except Exception as ex:
        print(ex)

def Translate(targetLanguage):
    translation = ''

    # Translate speech
    # Translate speech
    # audio_config = speech_sdk.AudioConfig(use_default_microphone=True)
    # translator = speech_sdk.translation.TranslationRecognizer(translation_config, audio_config)
    # print("Speak now...")
    # result = translator.recognize_once_async().get()
    # print('Translating "{}"'.format(result.text))
    # translation = result.translations[targetLanguage]
    # print(translation)

    # Translate speech
    audioFile = 'station.wav'
    playsound(audioFile)
    audio_config = speech_sdk.AudioConfig(filename=audioFile)
    translator = speech_sdk.translation.TranslationRecognizer(translation_config, audio_config)
    print("Getting speech from file...")
    result = translator.recognize_once_async().get()
    print('Translating "{}"'.format(result.text))
    translation = result.translations[targetLanguage]
    print(translation)

    # Synthesize translation
    # Synthesize translation
    voices = {
            "fr": "fr-FR-HenriNeural",
            "es": "es-ES-ElviraNeural",
            "hi": "hi-IN-MadhurNeural"
    }
    speech_config.speech_synthesis_voice_name = voices.get(targetLanguage)
    speech_synthesizer = speech_sdk.SpeechSynthesizer(speech_config)
    speak = speech_synthesizer.speak_text_async(translation).get()
    if speak.reason != speech_sdk.ResultReason.SynthesizingAudioCompleted:
        print(speak.reason)


if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'dotenv'

<h3>Create a Language Understanding app</h3>

    Provision Azure resources for the Language Understanding service.
    Define intents and utterances.
    Define entities.
    Use patterns to differentiate similar utterances.
    Use pre-built models.
    Train, test, publish, and review a model.
    
To use the Language Understanding service to develop a natural language understanding solution, you require two kinds of resource in your Azure subscription:

1. An Authoring resource (which must be a Language Understanding - Authoring resource) that you can use to train your language understanding model.
2. A Prediction resource (which can be a Language Understanding - Prediction or Cognitive Services resource) to host your trained model and process requests from client applications.

The Prediction resource prioritizes the client requests by giving you more prediction endpoint requests per month than the Authoring resource for the purposes of supporting client applications.

Utterances are the phrases that a user might enter when interacting with an application that uses your Language Understanding model. An intent represents a task or action the user wants to perform, or more simply the meaning of an utterance. You create a model by defining intents and associating them with one or more utterances.

In addition to the intents that you define, every model includes None intent that you should use to explicitly identify utterances that a user might submit, but for which there is no specific action required (for example, conversational greetings like "hello") or that fall outside of the scope of the domain for this model.

Entities are used to add specific context to intents. For example, you might define a TurnOnDevice intent that can be applied to multiple devices, and use entities to define the different devices.

You can define entities in a number of ways:

    1. Machine learned entities are the most flexible kind of entity, and should be used in most cases. You define a machine learned entity with a suitable name, and then associate words or phrases with it in training utterances. When you train your model, it learns to match the appropriate elements in the utterances with the entity.
    2. List entities are useful when you need an entity with a specific set of possible values - for example, days of the week. You can include synonyms in a list entity definition, so you could define a DayOfWeek entity that includes the values "Sunday", "Monday", "Tuesday", and so on; each with synonyms like "Sun", "Mon", "Tue", and so on.
    3. Regular Expression or RegEx entities are useful when an entity can be identified by matching a particular format of string. For example, a date in the format MM/DD/YYYY, or a flight number in the format AB-1234.
    4. Pattern.any() entities are used with patterns, which are discussed in the next topic.

<h3>Use patterns to differentiate similar utterances</h3>

You could associate utterances for every possible entity with all three intents. However, a more efficient way to train the model is to define patterns that include utterance templates, like this:

TurnOnDevice:
    1. "Turn the {DeviceName} on."
    2. "Switch the {DeviceName} on."
    3. "Turn on the {DeviceName}."
GetDeviceStatus:
    1. "Is the {DeviceName} on[?]"
TurnOffDevice:
    1. "Turn the {DeviceName} off."
    2. "Switch the {DeviceName} off."
    3. "Turn off the {DeviceName}."
These utterances include a placeholder for a Pattern.any() entity named DeviceName. reducing the number of utterances required to train the model. Patterns can make use of optional elements, such as punctuation to provide additional cues that help identify the appropriate intent.

Authouring resourse has to be a Language Understanding- Authouring<br>
However, prediction resource can be a LU or cognitive service resource

<h3>Set publishing configuration options</h3>

Every Language Understanding app has two publishing slots:

    Staging. Use this slot to publish and test new versions of your language model without disrupting production applications.
    Production. Use this slot for "live" models that are used by production applications.

<h3>Publish settings</h3>

Regardless of which slot you publish your Language Understanding app to, you can configure the following publish settings to enable specific behavior:

    Sentiment analysis. Enable this to include a sentiment score from 0 (negative) to 1 (positive) in predictions. This score reflects the sentiment of the input utterance.
    Spelling correction. Enable this to use the Bing Spell Check service to correct the spelling on input utterances before intent prediction.
    Speech priming. Enable this if you plan to use the language model with the Speech service. This option sends the model to the Speech service ahead of prediction to improve intent recognition from spoken input.
    
To consume your Language Understanding model in a client application, you can use the REST APIs or one of the programming language-specific SDKs.

Regardless of the approach used, requests for predictions are sent to a published slot (production or staging) and include the following parameters:

    query - the utterance text to be analyzed.
    show-all-intents - indicates whether to include all identified intents and their scores, or only the most likely intent.
    verbose - used to include additional metadata in the results, such as the start index and length of strings identified as entities,
    log used to record queries and results for use in Active Learning.

<h3>Prediction results</h3>

A typical response might look similar to this:


In [1]:
{
  "query": "What's the time in Edinburgh?",
  "prediction": {
    "topIntent": "GetTime",
    "intents": {
      "GetTime": {
        "score": 0.9978
      },
      ...
    },
    "entities": {
      "location": ["Edinburgh"],
      ...
    }
  }
}

SyntaxError: invalid syntax (<ipython-input-1-2d6c2dae1deb>, line 10)

<h3>Use a container</h3>

Like many other cognitive services, the Language Understanding service can also be deployed as a container, running in a local Docker host, an Azure Container Instance (ACI), or in an Azure Kubernetes Service (AKS) cluster.

1. Download the container image
2. Export the model for a container
3. Run the container with required parameters:
        Prediction endpoint for billing
        Prediction key
        EULA acceptance
        Mount points (input for exported model, output for logs)
4. Use the container to predict intents for client apps
5. The container sends usage metrics to your prediction resource for billing

steps: pull docker image -> export Language understanding app in .gz format -> run docker container with image and mount input folder with the exported model

<h3>Understand Language Understanding and Speech service integration</h3>

To use the Speech SDK with a Language Understanding model, enable the Speech priming publishing setting for your Language Understanding endpoint, and use the Speech SDK to write code that uses your Language Understanding prediction resource

To use a Language Understanding model from the Speech SDK, your code should follow this pattern:

1. Use a SpeechConfig object to encapsulate the information required to connect to your Language Understanding prediction resource (not a Speech resource). Specifically, the SpeechConfig must be configured with the location and key of the Language Understanding prediction resource.
2. Optionally, use an AudioConfig to define the input source for the speech to be analyzed. By default, this is the default system microphone, but you can also specify an audio file.
3. Use the SpeechConfig and AudioConfig to create an IntentRecognizer object, and add the model and the intents you want to recognize to its configuration.
4. Use the methods of the IntentRecognizer object to submit utterances to the Language understanding prediction endpoint. For example, the RecognizeOnceAsync() method submits a single spoken utterance.
5. Process the response. In the case of the RecognizeOnceAsync() method, the result is an IntentRecognitionResult object that includes the following properties:

        Duration
        IntendId
        OffsetInTicks
        Properties
        Reason
        ResultId
        Text

If the operation was successful, the Reason property has the enumerated value RecognizedIntent, and the IntentId property contains the top intent name. Full details of the Language Understanding prediction can be found in the Properties property, which includes the full JSON prediction.

Other possible values for Result include RecognizedSpeech, which indicates that the speech was successfully transcribed (the transcription is in the Text property), but no matching intent was identified. If the result is NoMatch, the audio was successfully parsed but no speech was recognized, and if the result is Canceled, an error occurred (in which case, you can check the Properties collection for the CancellationReason property to determine what went wrong.)

Enable Speech Priming while publishing your application on luis.ai

In [3]:
#speaking-clock-client.py

from dotenv import load_dotenv
import os
import json
from datetime import datetime, timedelta, date
from dateutil.parser import parse as is_date
import playsound
# Import namespaces
# Import namespaces
import azure.cognitiveservices.speech as speech_sdk

def main():

    try:
        # Get Configuration Settings
        load_dotenv()
        lu_app_id = os.getenv('LU_APP_ID')
        lu_prediction_region = os.getenv('LU_PREDICTION_REGION')
        lu_prediction_key = os.getenv('LU_PREDICTION_KEY')

        # Configure speech service and get intent recognizer
        # Configure speech service and get intent recognizer
        #speech_config = speech_sdk.SpeechConfig(subscription=lu_prediction_key, region=lu_prediction_region)
        #audio_config = speech_sdk.AudioConfig(use_default_microphone=True)
        #recognizer = speech_sdk.intent.IntentRecognizer(speech_config, audio_config)

        # Configure speech service and get intent recognizer
        audioFile = 'time-in-london.wav'
        playsound(audioFile)
        speech_config = speech_sdk.SpeechConfig(subscription=lu_prediction_key, region=lu_prediction_region)
        audio_config = speech_sdk.AudioConfig(filename=audioFile)
        recognizer = speech_sdk.intent.IntentRecognizer(speech_config, audio_config)

        # Get the model from the AppID and add the intents we want to use
        # Get the model from the AppID and add the intents we want to use
        model = speech_sdk.intent.LanguageUnderstandingModel(app_id=lu_app_id)
        intents = [
            (model, "GetTime"),
            (model, "GetDate"),
            (model, "GetDay"),
            (model, "None")
        ]
        recognizer.add_intents(intents)

        # Process speech input
        # Process speech input
        intent = ''
        result = recognizer.recognize_once_async().get()
        if result.reason == speech_sdk.ResultReason.RecognizedIntent:
            intent = result.intent_id
            print("Query: {}".format(result.text))
            print("Intent: {}".format(intent))
            json_response = json.loads(result.intent_json)
            print("JSON Response:\n{}\n".format(json.dumps(json_response, indent=2)))

            # Get the first entity (if any)
            # Get the first entity (if any)
            entity_type = ''
            entity_value = ''
            if len(json_response["entities"]) > 0:
                entity_type = json_response["entities"][0]["type"]
                entity_value = json_response["entities"][0]["entity"]
                print(entity_type + ': ' + entity_value)
            # Apply the appropriate action
            # Apply the appropriate action
            if intent == 'GetTime':
                location = 'local'
                # Check for entities
                if entity_type == 'Location':
                    location = entity_value
                # Get the time for the specified location
                print(GetTime(location))

            elif intent == 'GetDay':
                date_string = date.today().strftime("%m/%d/%Y")
                # Check for entities
                if entity_type == 'Date':
                    date_string = entity_value
                # Get the day for the specified date
                print(GetDay(date_string))

            elif intent == 'GetDate':
                day = 'today'
                # Check for entities
                if entity_type == 'Weekday':
                    # List entities are lists
                    day = entity_value
                # Get the date for the specified day
                print(GetDate(day))

            else:
                # Some other intent (for example, "None") was predicted
                print('You said {}'.format(result.text))
                if result.text.lower().replace('.', '') == 'stop':
                    intent = result.text
                else:
                    print('Try asking me for the time, the day, or the date.')


        elif result.reason == speech_sdk.ResultReason.RecognizedSpeech:
            # Speech was recognized, but no intent was identified.
            intent = result.text
            print("I don't know what {} means.".format(intent))
        elif result.reason == speech_sdk.ResultReason.NoMatch:
            # Speech wasn't recognized
            print("Sorry. I didn't understand that.")
        elif result.reason == speech_sdk.ResultReason.Canceled:
            # Something went wrong
            print("Intent recognition canceled: {}".format(result.cancellation_details.reason))
            if result.cancellation_details.reason == speech_sdk.CancellationReason.Error:
                print("Error details: {}".format(result.cancellation_details.error_details))

    except Exception as ex:
        print(ex)


def GetTime(location):
    time_string = ''

    # Note: To keep things simple, we'll ignore daylight savings time and support only a few cities.
    # In a real app, you'd likely use a web service API (or write  more complex code!)
    # Hopefully this simplified example is enough to get the the idea that you
    # use LU to determine the intent and entitites, then implement the appropriate logic

    if location.lower() == 'local':
        now = datetime.now()
        time_string = '{}:{:02d}'.format(now.hour,now.minute)
    elif location.lower() == 'london':
        utc = datetime.utcnow()
        time_string = '{}:{:02d}'.format(utc.hour,utc.minute)
    elif location.lower() == 'sydney':
        time = datetime.utcnow() + timedelta(hours=11)
        time_string = '{}:{:02d}'.format(time.hour,time.minute)
    elif location.lower() == 'new york':
        time = datetime.utcnow() + timedelta(hours=-5)
        time_string = '{}:{:02d}'.format(time.hour,time.minute)
    elif location.lower() == 'nairobi':
        time = datetime.utcnow() + timedelta(hours=3)
        time_string = '{}:{:02d}'.format(time.hour,time.minute)
    elif location.lower() == 'tokyo':
        time = datetime.utcnow() + timedelta(hours=9)
        time_string = '{}:{:02d}'.format(time.hour,time.minute)
    elif location.lower() == 'delhi':
        time = datetime.utcnow() + timedelta(hours=5.5)
        time_string = '{}:{:02d}'.format(time.hour,time.minute)
    else:
        time_string = "I don't know what time it is in {}".format(location)
    
    return time_string

def GetDate(day):
    date_string = 'I can only determine dates for today or named days of the week.'

    weekdays = {
        "monday":0,
        "tuesday":1,
        "wednesday":2,
        "thusday":3,
        "friday":4,
        "saturday":5,
        "sunday":6
    }

    today = date.today()

    # To keep things simple, assume the named day is in the current week (Sunday to Saturday)
    day = day.lower()
    if day == 'today':
        date_string = today.strftime("%m/%d/%Y")
    elif day in weekdays:
        todayNum = today.weekday()
        weekDayNum = weekdays[day]
        offset = weekDayNum - todayNum
        date_string = (today + timedelta(days=offset)).strftime("%m/%d/%Y")

    return date_string

def GetDay(date_string):
    # Note: To keep things simple, dates must be entered in US format (MM/DD/YYYY)
    try:
        date_object = datetime.strptime(date_string, "%m/%d/%Y")
        day_string = date_object.strftime("%A")
    except:
        day_string = 'Enter a date in MM/DD/YYYY format.'
    return day_string

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'dotenv'

<h3>Create QnA application</h3>

To create a knowledge base:

1. Create a Language resource in your Azure subscription.
    Enable the question answering feature.
    Create or select an Azure Cognitive Search resource to host the knowledge base index.
2. In Language Studio, select the Language resource and create a Custom question answering project.
3. Name the knowledge base.
4. Add one or more data sources to populate the knowledge base:
    URLs for web pages containing FAQs.
    Files containing structured text from which questions and answers can be derived.
    Pre-defined chit-chat datasets that include common conversational questions and responses in a specified style.
5. Create the knowledge base and edit question and answer pairs in the portal.

You can enable multi-turn responses when importing questions and answers from an existing web page or document based on its structure, or you can explicitly define follow-up prompts and responses for existing question and answer pairs.

For example, suppose an initial question for a travel booking knowledge base is "How can I cancel a reservation?". A reservation might refer to a hotel or a flight, so a follow-up prompt is required to clarify this detail. The answer might consist of text such as "Cancellation policies depend on the type of reservation" and include follow-up prompts with links to answers about canceling flights and canceling hotels.

You can test your knowledge base interactively in Language Studio, submitting questions and reviewing the answers that are returned. 

When you are happy with the performance of your knowledge base, you can deploy it to a REST endpoint that client applications can use to submit questions and receive answers.

To consume the published knowledge base, you can use the REST interface.

The minimal request body for the function contains a question, like this:    

In [4]:
{
  "question": "What do I need to do to cancel a reservation?"
}

{'question': 'What do I need to do to cancel a reservation?'}

The response includes the closest question match that was found in the knowledge base, along with the associated answer, the confidence score, and other metadata about the question and answer pair.

In [5]:
{
  "answers": [
    {
      "questions": [
        "How can I cancel a reservation?"
      ],
      "answer": "Call us on 555 123 4567 to cancel a reservation.",
      "confidenceScore": 1.0,
      "id": 6,
      "source": "https://margies-travel.com/faq",
      "metadata": {},
      "dialog": {
        "isContextOnly": false,
        "prompts": []
      }
    }
  ]
}

NameError: name 'false' is not defined

Select the ＋Create a resource button, search for Language, and create a Language service resource.

Click Select on the Custom question answering block. Then click Continue to create your resource.

*Custom Question Answering uses Azure Search to index and query the knowledge base of questions and answers.

<h3>Create a bot using bot framework sdk</h3>

Bot solutions on Microsoft Azure are supported by the following technologies:
    1. Azure Bot Service. A cloud service that enables bot delivery through one or more channels, and integration with other services.
    2. Bot Framework Service. A component of Azure Bot Service that provides a REST API for handling bot activities.
    3. Bot Framework SDK. A set of tools and libraries for end-to-end bot development that abstracts the REST interface, enabling bot development in a range of programming languages.

The Bot Framework SDK provides an extensive set of tools and libraries that software engineers can use to develop bots.

<h3>Bot templates</h3>

The easiest way to get started with the Bot Framework SDK is to base your new bot on one the templates it provides:

    Empty Bot - a basic bot skeleton.
    Echo Bot - a simple "hello world" sample in which the bot responds to messages by echoing the message text back to the user.
    Core Bot - a more comprehensive bot that includes common bot functionality, such as integration with the Language Understanding service.

Bot application classes and logic:
    
The template bots are based on the Bot class defined in the Bot Framework SDK, which is used to implement the logic in your bot that receives and interprets user input, and responds appropriately. Additionally, bots make use of an Adapter class that handles communication with the user's channel.

Conversations in a bot are composed of activities, which represent events such as a user joining a conversation or a message being received. These activities occur within the context of a turn, a two-way exchange between the user and bot. The Bot Framework Service notifies your bot's adapter when an activity occurs in a channel by calling its Process Activity method, and the adapter creates a context for the turn and calls the bot's Turn Handler method to invoke the appropriate logic for the activity.

The Bot Framework Emulator is an application that enables you to run your bot a local or remote web application and connect to it from an interactive web chat interface that you can use to test your bot

<h3>Implement activity handlers and dialogs</h3>

The logic for processing the activity can be implemented in multiple ways. The Bot Framework SDK provides classes that can help you build bots that manage conversations using:

    Activity handlers: Event methods that you can override to handle different kinds of activities.
    Dialogs: More complex patterns for handling stateful, multi-turn conversations.

Activity handlers

For simple bots with short, stateless interactions, you can use Activity Handlers to implement an event-driven conversation model in which the events are triggered by activities such as users joining the conversation or a message being received. When an activity occurs in a channel, the Bot Framework Service calls the bot adapter's Process Activity function, passing the activity details. The adapter creates a turn context for the activity and passes it to the bot's turn handler, which calls the individual, event-specific activity handler.

The ActivityHandler base class includes event methods for the many kinds of common activity, including:

    Message received
    Members joined the conversation
    Members left the conversation
    Message reaction received
    Bot installed
    Others...

Turn context

An activity occurs within the context of a turn, which represents a single two-way exchange between the user and the bot. Activity handler methods include a parameter for the turn context, which you can use to access relevant information. For example, the activity handler for a message received activity includes the text of the message.

Dialogs

For more complex conversational flows where you need to store state between turns to enable a multi-turn conversation, you can implement dialogs. The Bot Framework SDK dialogs library provides multiple dialog classes that you can combine to implement the required conversational flow for your bot.

There are two common patterns for using dialogs to compose a bot conversation:
    1. Component dialogs: A component dialog is a dialog that can contain other dialogs, defined in its dialog set. Often, the initial dialog in the component dialog is a waterfall dialog
    2. Adaptive dialogs: An adaptive dialog is another kind of container dialog in which the flow is more flexible, allowing for interruptions, cancellations, and context switches at any point in the conversation. In this style of conversation, the bot initiates a root dialog, which contains a flow of actions (which can include branches and loops), and triggers that can be initiated by actions or by a recognize
